In [61]:
#Importing the necessary Libraries
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [62]:
#as we have data in directries which has further sub-directories in the json format we have read them all in the same format
def get_data_files(filepath):
    all_files= []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
        
    return all_files
    
    

Processing song data

In [76]:
#all the songs data will stored in this 
song_files = get_data_files("data\song_data")
song_files

['c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json',
 'c:\\Users\\Muhamma

In [91]:
import json


for file_path in song_files[:5]:
    print("File Path:", file_path)
    with open(file_path, 'r') as file:
        data = json.load(file)
        print("Data:", data)
    print()

File Path: c:\Users\Muhammad Arham\Desktop\DataEng_Proj\Data Modeling\Modeling_Postgresql\data\song_data\A\A\A\TRAAAAW128F429D538.json
Data: {'num_songs': 1, 'artist_id': 'ARD7TVE1187B99BFB1', 'artist_latitude': None, 'artist_longitude': None, 'artist_location': 'California - LA', 'artist_name': 'Casual', 'song_id': 'SOMZWCG12A8C13C480', 'title': "I Didn't Mean To", 'duration': 218.93179, 'year': 0}

File Path: c:\Users\Muhammad Arham\Desktop\DataEng_Proj\Data Modeling\Modeling_Postgresql\data\song_data\A\A\A\TRAAABD128F429CF47.json
Data: {'num_songs': 1, 'artist_id': 'ARMJAGH1187FB546F3', 'artist_latitude': 35.14968, 'artist_longitude': -90.04892, 'artist_location': 'Memphis, TN', 'artist_name': 'The Box Tops', 'song_id': 'SOCIWDW12A8C13D406', 'title': 'Soul Deep', 'duration': 148.03546, 'year': 1969}

File Path: c:\Users\Muhammad Arham\Desktop\DataEng_Proj\Data Modeling\Modeling_Postgresql\data\song_data\A\A\A\TRAAADZ128F9348C2E.json
Data: {'num_songs': 1, 'artist_id': 'ARKRRTF1187B9

In [100]:
# Initialize an empty list to store DataFrames
dfs= []

# Iterate over each file path
for filepath in song_files:
    # Check if the file exists
    if os.path.exists(filepath):
        # Read the JSON file into a DataFrame
        df = pd.read_json(filepath, dtype={'year': int}, lines=True)
        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print(f"File not found: {filepath}")

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# Display the first few records of the combined DataFrame
print(df.head())

   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   
1          1  ARMJAGH1187FB546F3         35.14968         -90.04892   
2          1  ARKRRTF1187B9984DA              NaN               NaN   
3          1  AR7G5I41187FB4CE6C              NaN               NaN   
4          1  ARXR32B1187FB57099              NaN               NaN   

   artist_location       artist_name             song_id             title  \
0  California - LA            Casual  SOMZWCG12A8C13C480  I Didn't Mean To   
1      Memphis, TN      The Box Tops  SOCIWDW12A8C13D406         Soul Deep   
2                   Sonora Santanera  SOXVLOJ12AB0189215   Amor De Cabaret   
3  London, England          Adam Ant  SONHOTT12A8C13493C   Something Girls   
4                                Gob  SOFSOCN12A8C143F5D    Face the Ashes   

    duration  year  
0  218.93179     0  
1  148.03546  1969  
2  177.47546     0  
3  233.40363  1982  

In [108]:
#lets separate the attributes for the "songs" table

song_data = df[['song_id','title','year','duration', 'artist_id']]
song_data.head(3)

,song_id,title,year,duration,artist_id
0,SOMZWCG12A8C13C480,I Didn't Mean To,0,218.93179,ARD7TVE1187B99BFB1
1,SOCIWDW12A8C13D406,Soul Deep,1969,148.03546,ARMJAGH1187FB546F3
2,SOXVLOJ12AB0189215,Amor De Cabaret,0,177.47546,ARKRRTF1187B9984DA


In [119]:
#lets separate the attributes for the "artist" table
artist_data = df[['artist_id','artist_latitude','artist_longitude','artist_location', 'artist_name']].values.tolist()
artist_data[:2]

[['ARD7TVE1187B99BFB1', nan, nan, 'California - LA', 'Casual'],
 ['ARMJAGH1187FB546F3', 35.14968, -90.04892, 'Memphis, TN', 'The Box Tops']]

Processing Log data

In [68]:
log_files = get_data_files("data\log_data")
log_files

['c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-01-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-02-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-03-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-04-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-05-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-06-events.json',
 'c:\\Users\\Muhammad Arham\\Desktop\\DataEng_Proj\\Data Modeling\\Modeling_Postgresql\\data\\log_data\\2018\\11\\2018-11-07-events.json',
 'c:\\Users\\Muhammad Arham

In [89]:
file_path_log = log_files[0]


In [90]:
df_log = pd.read_json(file_path_log, lines= True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


User data


In [116]:
user_data = df_log[['userId','userAgent','firstName','lastName', 'gender', 'level']].values.tolist()
user_data[:2]

[[39,
  '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"',
  'Walter',
  'Frye',
  'M',
  'free'],
 [8,
  '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"',
  'Kaylee',
  'Summers',
  'F',
  'free']]

Timestamp

In [120]:
time_stamp = pd.to_datetime(df_log["ts"], unit='ms')
time_stamp.head()

0   2018-11-01 20:57:10.796
1   2018-11-01 21:01:46.796
2   2018-11-01 21:01:46.796
3   2018-11-01 21:02:12.796
4   2018-11-01 21:05:52.796
Name: ts, dtype: datetime64[ns]

In [121]:
time_data = (time_stamp, time_stamp.dt.hour, time_stamp.dt.day, time_stamp.dt.week, time_stamp.dt.month, time_stamp.dt.year, time_stamp.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

C:\Users\Muhammad Arham\AppData\Local\Temp\ipykernel_12532\2567860215.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  time_data = (time_stamp, time_stamp.dt.hour, time_stamp.dt.day, time_stamp.dt.week, time_stamp.dt.month, time_stamp.dt.year, time_stamp.dt.weekday)


,start_time,hour,day,week,month,year,weekday
0,2018-11-01 20:57:10.796,20,1,44,11,2018,3
1,2018-11-01 21:01:46.796,21,1,44,11,2018,3
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
3,2018-11-01 21:02:12.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
